In [1]:
import numpy as np 
import pandas as pd
import random
from scipy import stats
from ast import literal_eval
import warnings
warnings.filterwarnings('ignore')

import networkx as nx
import obonet

from itertools import product

In [2]:
def cliffs_delta(x, y):
    """
    Compute Cliff's Delta effect size.
    """
    x, y = np.array(x), np.array(y)
    n_x, n_y = len(x), len(y)
    
    N_plus = sum(1 for a, b in product(x, y) if a > b)
    N_minus = sum(1 for a, b in product(x, y) if a < b)
    
    delta = (N_plus - N_minus) / (n_x * n_y)

    return delta#, interpretation

def bootstrapping_effect_size(a, b, nsteps = 100000):
    es_mean = cliffs_delta(a, b)
    n1, n2 = len(a), len(b)
    es_bootstrap = np.zeros(nsteps)
    for i in range(nsteps):
        a_random = np.random.choice(a, size=n1)
        b_random = np.random.choice(b, size=n2)
        cliff_b = cliffs_delta(a_random, b_random)
        es_bootstrap[i] = cliff_b
#         print(f"{i}: {cliff_b:.4f}")
    es_lower = np.percentile(es_bootstrap, 2.5)
    es_upper = np.percentile(es_bootstrap, 97.5)
    es_se = np.std(es_bootstrap)
    return es_mean, es_se, (es_lower, es_upper)
    


In [3]:
# This is the dataset that excluded chain shorter than 100 residues
df_idrome = pd.read_pickle('../IDRome_DB_reanalyzed_entanglement_N_ge_100.pkl')
# This is the GO term associated to each uniprot downloaded from Uniprot. Need to update this
uniprot2go_dict = dict(pd.read_pickle('uniprot_dict_Quyen_call.pkl'))
df_GO = pd.DataFrame({'Uniprot': uniprot2go_dict.keys(), 'GO':uniprot2go_dict.values()})

GO_dict = pd.Series(pd.read_pickle('uniprot_dict_Quyen_call.pkl')).to_frame(name='GO')
# count number of occurance of each GO term in this dataset
g,u = np.unique(GO_dict.loc[df_idrome.UniProt_ID].GO.sum(),return_counts=True)
# only consider GO term apprear more than 200 times
curated_GO_codes = g[u>=200]

print(f"Total number of GO term need to be analyzed: {len(curated_GO_codes)}")

Total number of GO term need to be analyzed: 91


In [4]:
url = 'http://purl.obolibrary.org/obo/go/go-basic.obo'
graph = obonet.read_obo(url)

id_to_name = {id_: data.get('name') for id_, data in graph.nodes(data=True)}
name_to_id = {data['name']: id_ for id_, data in graph.nodes(data=True) if 'name' in data}

In [5]:
# extract molecular function, biological function and celluler component go term
print(name_to_id['molecular_function'],name_to_id['biological_process'],name_to_id['cellular_component'])
molfunc = list(nx.ancestors(graph, 'GO:0003674')) # all nodes connected to molecular function
bioproc = list(nx.ancestors(graph, 'GO:0008150')) # all nodes connected to biological process
cellcom = list(nx.ancestors(graph, 'GO:0005575')) # all nodes connected to cellular component

GO:0003674 GO:0008150 GO:0005575


In [6]:
molfunc_go_code = set(curated_GO_codes).intersection(set(molfunc))
bioproc_go_code = set(curated_GO_codes).intersection(set(bioproc))
cellcom_go_code = set(curated_GO_codes).intersection(set(cellcom))

# Molecular function

Notes:
- The Brunner Mulzel in Scipy calculate the statsistic (x, y) as <rankcy> - <rankcx>. If the statistic is negative number, indicate member in x is larger than y

In [7]:
"""
Loop through all row of df_GO and check if any GO code of the current uniprot are in list of molecular function GO
code, if yes, store in the tuple for grouping by GO code later.
"""
filtered_data = []

for _, row in df_GO.iterrows():
    for c in row['GO']:
        if c in molfunc_go_code:
            filtered_data.append((c, row['Uniprot']))

df_result = (
    pd.DataFrame(filtered_data, columns=["GO", "Uniprot"])
    .groupby("GO")["Uniprot"]
    .apply(list)
    .reset_index()
)
GO_terms = []
ent_prob_GO_Yes = []
ent_prob_GO_No = []
GO_statistic = []
effect_size = []
effect_size_se = []
effect_size_cis = []
pvalue_go_term = []
for _, row in df_result.iterrows():
    go, uniprots = row['GO'], row['Uniprot']
    
    df_idrome_go = df_idrome.copy()
    df_idrome_go['GO_term'] = df_idrome_go.UniProt_ID.apply(lambda x: True if x in uniprots else False)
    # Extract all uniprot code that are associated with the current GO code, then look back in df_idrome dataframe
    # to extract the entanglement probability, convert to list.
    GO_Yes = np.array(df_idrome_go[df_idrome_go['GO_term']==True]['ent_prob'].to_list())
    GO_No = np.array(df_idrome_go[df_idrome_go['GO_term']==False]['ent_prob'].to_list())
    ent_prob_GO_Yes.append(GO_Yes)
    ent_prob_GO_No.append(GO_No)

    statistic, pvalues = stats.brunnermunzel(GO_Yes, GO_No, alternative='two-sided')
    es_mean, es_se, es_ci = bootstrapping_effect_size(GO_Yes, GO_No, nsteps=1)
    
    GO_terms.append(go)
    GO_statistic.append(statistic)
    effect_size.append(es_mean)
    effect_size_se.append(es_se)
    effect_size_cis.append(es_ci)
    pvalue_go_term.append(pvalues/2.0)

df_BM_analysis = pd.DataFrame({'GO':GO_terms, 'ent_prob_GO_Yes': ent_prob_GO_Yes, 'ent_prob_GO_No':ent_prob_GO_No, 'effect_size': effect_size, 'effect_size_se': effect_size_se, 'effect_size_CI':effect_size_cis, 'stats':GO_statistic, 'pvalue':pvalue_go_term})
df_BM_analysis['adj_pvalue'] = stats.false_discovery_control(df_BM_analysis['pvalue'])
for _, row in df_BM_analysis[df_BM_analysis['adj_pvalue']<=0.05].iterrows():
    print(f"{row['GO']}:{id_to_name[row['GO']]}, {row['adj_pvalue']:.3f}, stats: {row['stats']:.3f}, effect size: {row['effect_size']:.3f}+/-{row['effect_size_se']:.4f}, {row['effect_size_CI']}")
    

# save to pickle file
df_BM_analysis.to_pickle('molfunc.pkl')

GO:0000978:RNA polymerase II cis-regulatory region sequence-specific DNA binding, 0.000, stats: -3.988, effect size: 0.065+/-0.0000, (np.float64(0.06269040989296622), np.float64(0.06269040989296622))
GO:0001227:DNA-binding transcription repressor activity, RNA polymerase II-specific, 0.021, stats: -2.688, effect size: 0.084+/-0.0000, (np.float64(0.06460698244237773), np.float64(0.06460698244237773))
GO:0001228:DNA-binding transcription activator activity, RNA polymerase II-specific, 0.044, stats: -2.319, effect size: 0.058+/-0.0000, (np.float64(0.0687986585132991), np.float64(0.0687986585132991))
GO:0003677:DNA binding, 0.014, stats: -2.873, effect size: 0.061+/-0.0000, (np.float64(0.06397026548980239), np.float64(0.06397026548980239))
GO:0003682:chromatin binding, 0.006, stats: -3.188, effect size: 0.090+/-0.0000, (np.float64(0.11255976713191278), np.float64(0.11255976713191278))
GO:0019899:enzyme binding, 0.035, stats: 2.464, effect size: -0.100+/-0.0000, (np.float64(-0.1139673719172

# Biological Process

In [8]:
"""
Loop through all row of df_GO and check if any GO code of the current uniprot are in list of molecular function GO
code, if yes, store in the tuple for grouping by GO code later.
"""
filtered_data = []

for _, row in df_GO.iterrows():
    for c in row['GO']:
        if c in bioproc_go_code:
            filtered_data.append((c, row['Uniprot']))

df_result = (
    pd.DataFrame(filtered_data, columns=["GO", "Uniprot"])
    .groupby("GO")["Uniprot"]
    .apply(list)
    .reset_index()
)
GO_terms = []
ent_prob_GO_Yes = []
ent_prob_GO_No = []
GO_statistic = []
effect_size = []
effect_size_se = []
effect_size_cis = []
pvalue_go_term = []
for _, row in df_result.iterrows():
    go, uniprots = row['GO'], row['Uniprot']
    
    df_idrome_go = df_idrome.copy()
    df_idrome_go['GO_term'] = df_idrome_go.UniProt_ID.apply(lambda x: True if x in uniprots else False)
    # Extract all uniprot code that are associated with the current GO code, then look back in df_idrome dataframe
    # to extract the entanglement probability, convert to list.
    GO_Yes = np.array(df_idrome_go[df_idrome_go['GO_term']==True]['ent_prob'].to_list())
    GO_No = np.array(df_idrome_go[df_idrome_go['GO_term']==False]['ent_prob'].to_list())
    ent_prob_GO_Yes.append(GO_Yes)
    ent_prob_GO_No.append(GO_No)

    statistic, pvalues = stats.brunnermunzel(GO_Yes, GO_No, alternative='two-sided')
    es_mean, es_se, es_ci = bootstrapping_effect_size(GO_Yes, GO_No, nsteps=1)
    
    GO_terms.append(go)
    GO_statistic.append(statistic)
    effect_size.append(es_mean)
    effect_size_se.append(es_se)
    effect_size_cis.append(es_ci)
    pvalue_go_term.append(pvalues/2.0)

df_BM_analysis = pd.DataFrame({'GO':GO_terms, 'ent_prob_GO_Yes': ent_prob_GO_Yes, 'ent_prob_GO_No':ent_prob_GO_No, 'effect_size': effect_size, 'effect_size_se': effect_size_se, 'effect_size_CI':effect_size_cis, 'stats':GO_statistic, 'pvalue':pvalue_go_term})
df_BM_analysis['adj_pvalue'] = stats.false_discovery_control(df_BM_analysis['pvalue'])
for _, row in df_BM_analysis[df_BM_analysis['adj_pvalue']<=0.05].iterrows():
    print(f"{row['GO']}:{id_to_name[row['GO']]}, {row['adj_pvalue']:.3f}, stats: {row['stats']:.3f}, effect size: {row['effect_size']:.3f}+/-{row['effect_size_se']:.4f}, {row['effect_size_CI']}")
    

# save to pickle file
df_BM_analysis.to_pickle('bioproc.pkl')

GO:0000122:negative regulation of transcription by RNA polymerase II, 0.015, stats: -2.793, effect size: 0.059+/-0.0000, (np.float64(0.07664788371114539), np.float64(0.07664788371114539))
GO:0006357:regulation of transcription by RNA polymerase II, 0.005, stats: -3.432, effect size: 0.052+/-0.0000, (np.float64(0.050132653695822316), np.float64(0.050132653695822316))
GO:0007155:cell adhesion, 0.037, stats: -2.416, effect size: 0.077+/-0.0000, (np.float64(0.08901359769841209), np.float64(0.08901359769841209))
GO:0015031:protein transport, 0.005, stats: 3.349, effect size: -0.138+/-0.0000, (np.float64(-0.10695798655848453), np.float64(-0.10695798655848453))
GO:0045893:positive regulation of DNA-templated transcription, 0.037, stats: -2.343, effect size: 0.059+/-0.0000, (np.float64(-0.006467042344395534), np.float64(-0.006467042344395534))
GO:0045944:positive regulation of transcription by RNA polymerase II, 0.005, stats: -3.253, effect size: 0.062+/-0.0000, (np.float64(0.03679827909893286

# Cellular Compartment

In [9]:
"""
Loop through all row of df_GO and check if any GO code of the current uniprot are in list of molecular function GO
code, if yes, store in the tuple for grouping by GO code later.
"""
filtered_data = []

for _, row in df_GO.iterrows():
    for c in row['GO']:
        if c in cellcom_go_code:
            filtered_data.append((c, row['Uniprot']))

df_result = (
    pd.DataFrame(filtered_data, columns=["GO", "Uniprot"])
    .groupby("GO")["Uniprot"]
    .apply(list)
    .reset_index()
)
GO_terms = []
ent_prob_GO_Yes = []
ent_prob_GO_No = []
GO_statistic = []
effect_size = []
effect_size_se = []
effect_size_cis = []
pvalue_go_term = []
for _, row in df_result.iterrows():
    go, uniprots = row['GO'], row['Uniprot']
    
    df_idrome_go = df_idrome.copy()
    df_idrome_go['GO_term'] = df_idrome_go.UniProt_ID.apply(lambda x: True if x in uniprots else False)
    # Extract all uniprot code that are associated with the current GO code, then look back in df_idrome dataframe
    # to extract the entanglement probability, convert to list.
    GO_Yes = np.array(df_idrome_go[df_idrome_go['GO_term']==True]['ent_prob'].to_list())
    GO_No = np.array(df_idrome_go[df_idrome_go['GO_term']==False]['ent_prob'].to_list())
    ent_prob_GO_Yes.append(GO_Yes)
    ent_prob_GO_No.append(GO_No)

    statistic, pvalues = stats.brunnermunzel(GO_Yes, GO_No, alternative='two-sided')
    es_mean, es_se, es_ci = bootstrapping_effect_size(GO_Yes, GO_No, nsteps=1)
    
    GO_terms.append(go)
    GO_statistic.append(statistic)
    effect_size.append(es_mean)
    effect_size_se.append(es_se)
    effect_size_cis.append(es_ci)
    pvalue_go_term.append(pvalues/2.0)

df_BM_analysis = pd.DataFrame({'GO':GO_terms, 'ent_prob_GO_Yes': ent_prob_GO_Yes, 'ent_prob_GO_No':ent_prob_GO_No, 'effect_size': effect_size, 'effect_size_se': effect_size_se, 'effect_size_CI':effect_size_cis, 'stats':GO_statistic, 'pvalue':pvalue_go_term})
df_BM_analysis['adj_pvalue'] = stats.false_discovery_control(df_BM_analysis['pvalue'])
for _, row in df_BM_analysis[df_BM_analysis['adj_pvalue']<=0.05].iterrows():
    print(f"{row['GO']}:{id_to_name[row['GO']]}, {row['adj_pvalue']:.3f}, stats: {row['stats']:.3f}, effect size: {row['effect_size']:.3f}+/-{row['effect_size_se']:.4f}, {row['effect_size_CI']}")
    

# save to pickle file
df_BM_analysis.to_pickle('cellcom.pkl')

GO:0000139:Golgi membrane, 0.000, stats: 4.156, effect size: -0.133+/-0.0000, (np.float64(-0.09954036781859983), np.float64(-0.09954036781859983))
GO:0005576:extracellular region, 0.027, stats: 2.306, effect size: -0.051+/-0.0000, (np.float64(-0.05475560560309505), np.float64(-0.05475560560309505))
GO:0005634:nucleus, 0.001, stats: -3.615, effect size: 0.041+/-0.0000, (np.float64(0.01939200030068759), np.float64(0.01939200030068759))
GO:0005654:nucleoplasm, 0.009, stats: -2.771, effect size: 0.035+/-0.0000, (np.float64(0.03800663245497729), np.float64(0.03800663245497729))
GO:0005739:mitochondrion, 0.000, stats: 3.954, effect size: -0.112+/-0.0000, (np.float64(-0.1612942871068763), np.float64(-0.1612942871068763))
GO:0005783:endoplasmic reticulum, 0.000, stats: 4.239, effect size: -0.118+/-0.0000, (np.float64(-0.16866661709392175), np.float64(-0.16866661709392175))
GO:0005789:endoplasmic reticulum membrane, 0.030, stats: 2.240, effect size: -0.063+/-0.0000, (np.float64(-0.0793971941888